In [ ]:
import pandas as pd
import numpy as np
import os
from argparse import ArgumentParser
from Customize import JAVA_PATH, CPRD_CODE_PATH, COHORT_SAVE_PATH,MODEL_SAVE_PATH, get_symptom_medcode, get_icdcode
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from statannotations.Annotator import Annotator
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from statannotations.Annotator import Annotator

In [ ]:
parser = ArgumentParser()
parser.add_argument("--disease", type=str, default='AD')
parser.add_argument("--ukb_dir", type=str, default='UKB_AD_data')
parser.add_argument("--experient_dir", type=str, default='AD')
parser.add_argument("--model_name", type=str, default='cl_maskage_b32')
parser.add_argument("--stage", type=str, default='before')
parser.add_argument("--seed", type=int, default=2024)
parser.add_argument("--follow_up_year", type=int, default=5)
parser.add_argument("--k", type=int, default=5)

args = parser.parse_args([])
stage = args.stage
ukb_path = os.path.join(COHORT_SAVE_PATH, args.ukb_dir+'_'+args.stage)
experient_dir = os.path.join(MODEL_SAVE_PATH, args.experient_dir)
model_save_dir = os.path.join(experient_dir, args.model_name+'_'+stage)
results_dir = os.path.join(model_save_dir, 'results')
results_save_dir = os.path.join(results_dir, f'{args.disease}-{args.k}')
prs_save_dir = os.path.join(results_save_dir, 'prs')
if not os.path.exists(prs_save_dir):
    os.makedirs(prs_save_dir)

In [ ]:
prs_path = ''
prs = pd.read_csv(prs_path)
prs['patid'] = prs['patid'].astype(str)
result_df_ukb = pd.read_pickle(os.path.join(results_save_dir, 'result_df_'+stage+'_EHR_UKB.pkl'))
new_df = prs.merge(result_df_ukb[['patid', 'label']], on='patid', how='left')

In [ ]:
filtered_df = new_df.dropna(subset=['label'])
prs_columns = prs.columns[1:-1]
for col in prs_columns:
    if not pd.api.types.is_numeric_dtype(filtered_df[col]):
        try:
            filtered_df[col] = filtered_df[col].astype(float)
        except ValueError:
            print(f"Column {col} cannot be converted to float and will be excluded.")
            filtered_df.drop(columns=[col], inplace=True)

# Step 4: Calculate the mean PRS for each label
mean_prs_by_label = filtered_df.groupby('label').mean()

# Step 5: Filter PRS columns where the difference between max and min values across labels is greater than 0.3
columns_to_keep = mean_prs_by_label.columns[(mean_prs_by_label.max() - mean_prs_by_label.min()) > 0.15]

columns_to_keep = columns_to_keep.tolist()
filtered_mean_prs_by_label = mean_prs_by_label[columns_to_keep]
# Step 6: Create a heatmap without label and title, with increased font sizes
plt.figure(figsize=(30,20))
sns.heatmap(filtered_mean_prs_by_label.T, cmap='Blues', annot=True, fmt=".2f", 
            annot_kws={"size": 30},  # Increase font size of the cell values
            cbar=False,  # Remove the color bar
            xticklabels=True, yticklabels=True)

# Remove the title
plt.title('', fontsize=0)

# Increase the font size of the x-axis labels
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.xlabel('')
plt.ylabel('')
plt.tight_layout()
plt.savefig(os.path.join(prs_save_dir, 'prs_heatmap.png'))
plt.show()

In [ ]:
new_df['label'].fillna(0, inplace=True) # fill control people with 0
new_df['label'].replace({0: 'Control', 1: 'Cluster 1', 2: 'Cluster 2', 3: 'Cluster 3', 4: 'Cluster 4', 5: 'Cluster 5', 6: 'Cluster 6'}, inplace=True)
labels = new_df['label'].unique()
label_pairs = []
for (label1, label2) in itertools.combinations(labels, 2):
    label_pairs.append([label1, label2])

In [ ]:
from statannotations.Annotator import Annotator
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Exclude 'Control' and reset index
new_df_non = new_df[new_df['label'] != 'Control'].copy()
new_df_non.reset_index(drop=True, inplace=True)

clusters = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5']
prs_column = "Standard_PRS_for_alzheimer's_disease_AD"

# Define colors
cluster_colors = {
    'Cluster 1': '#1f77b4', 
    'Cluster 2': '#ff7f0e', 
    'Cluster 3': '#2ca02c', 
    'Cluster 4': '#d62728', 
    'Cluster 5': '#9467bd'
}
other_color = 'white'

# Create subplots
fig, axes = plt.subplots(1, 5, figsize=(25, 6), sharey=True)

for ax, cluster in zip(axes, clusters):
    # Define group column
    new_df_non['group'] = new_df_non['label'].apply(lambda x: cluster if x == cluster else 'Other')
    order = [cluster, 'Other']
    palette = {cluster: cluster_colors[cluster], 'Other': other_color}
    
    # Plot boxplot
    sns.boxplot(x='group',
                y=prs_column,
                data=new_df_non,
                order=order,
                palette=palette,
                ax=ax)
    
    # Welch's t-test
    group1 = new_df_non[new_df_non['group'] == cluster][prs_column]
    group2 = new_df_non[new_df_non['group'] == 'Other'][prs_column]
    t_stat, p_val = ttest_ind(group1, group2, equal_var=False)

    # Cohen's d
    mean1, mean2 = group1.mean(), group2.mean()
    std1, std2 = group1.std(), group2.std()
    pooled_sd = np.sqrt((std1**2 + std2**2) / 2)
    cohen_d = (mean1 - mean2) / pooled_sd if pooled_sd > 0 else np.nan
    
    # Annotator for p-value (star format)
    annotator = Annotator(ax,
                          pairs=[(cluster, 'Other')],
                          data=new_df_non,
                          x='group',
                          y=prs_column,
                          order=order)
    annotator.configure(test='t-test_welch',
                        text_format='star',
                        loc='inside',
                        line_offset=0.1,
                        verbose=0)
    annotator.apply_and_annotate()

    # Add effect size text manually
    ax.text(0.5,    # x position (between Cluster and Other)
            ax.get_ylim()[0] + 0.02,  # y position near bottom
            f"d = {cohen_d:.3f}", 
            ha='center', va='bottom', fontsize=10, fontstyle='italic')

    ax.set_title(f"{cluster} vs Other")

plt.tight_layout()
plt.savefig(os.path.join(prs_save_dir, f'UKB_{args.disease}_PRS_1vsother_with_effectsize.png'))
plt.show()

In [ ]:
# We use combinations from itertools to get all unique pairs of labels
for (label1, label2) in itertools.combinations(labels, 2):
    group1 = new_df[new_df['label'] == label1]['Standard_PRS_for_alzheimer\'s_disease_AD']
    group2 = new_df[new_df['label'] == label2]['Standard_PRS_for_alzheimer\'s_disease_AD']
    
    # Welch's t-test (assumes unequal variance)
    t_stat, p_val = ttest_ind(group1, group2, equal_var=False)
    
    results.append({
        'group1': label1,
        'group2': label2,
        't_stat': t_stat,
        'p_val': p_val
    })

# Convert results to a DataFrame
pairwise_results = pd.DataFrame(results)
pairwise_results

In [ ]:
import pandas as pd
import itertools
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import numpy as np

# Replace with your actual DataFrame
# new_df = pd.read_csv("your_input_file.csv")

# Initialize result list
results = []

# Unique cluster labels
labels = sorted(new_df['label'].unique())

# Loop through all unique pairs of labels
for (label1, label2) in itertools.combinations(labels, 2):
    group1 = new_df[new_df['label'] == label1]['Standard_PRS_for_alzheimer\'s_disease_AD']
    group2 = new_df[new_df['label'] == label2]['Standard_PRS_for_alzheimer\'s_disease_AD']

    # Welch's t-test
    t_stat, p_val = ttest_ind(group1, group2, equal_var=False)

    # Means and SDs for Cohen’s d
    mean1, mean2 = group1.mean(), group2.mean()
    sd1, sd2 = group1.std(), group2.std()
    n1, n2 = len(group1), len(group2)

    # Pooled standard deviation for unequal variances
    pooled_sd = np.sqrt(((sd1**2 + sd2**2) / 2))

    # Cohen's d (difference in means / pooled SD)
    cohen_d = (mean1 - mean2) / pooled_sd if pooled_sd > 0 else np.nan

    results.append({
        'group1': label1,
        'group2': label2,
        'mean1': mean1,
        'mean2': mean2,
        't_stat': t_stat,
        'p_val': p_val,
        'cohen_d': cohen_d
    })

# Create DataFrame
pairwise_results = pd.DataFrame(results)

# Apply Benjamini-Hochberg FDR correction
reject, p_corrected, _, _ = multipletests(pairwise_results['p_val'], alpha=0.05, method='fdr_bh')

# Add corrected p-values and significance flag
pairwise_results['p_val_adjusted'] = p_corrected
pairwise_results['significant'] = reject
pairwise_results.to_csv(os.path.join(prs_save_dir, f'UKB_{args.disease}_pairwise_PRS_results_with_effect_size.csv'), index=False)
pairwise_results = pairwise_results[pairwise_results['significant']==True]
pairwise_results.to_csv(os.path.join(prs_save_dir, f'UKB_{args.disease}_pairwise_PRS_results_with_effect_size_sig.csv'), index=False)

